## Dependencies

In [ ]:
#!pip install weaviate-client

  Cloning https://github.com/weaviate/weaviate-python-client.git (to revision fd9dbdda75b64d59ec7742a0e95157a9a608a0de) to /private/var/folders/20/sww521bs0s71v4pt86_05g340000gn/T/pip-req-build-9lgynotr
  Running command git clone --filter=blob:none --quiet https://github.com/weaviate/weaviate-python-client.git /private/var/folders/20/sww521bs0s71v4pt86_05g340000gn/T/pip-req-build-9lgynotr
  Running command git rev-parse -q --verify 'sha^fd9dbdda75b64d59ec7742a0e95157a9a608a0de'
  Running command git fetch -q https://github.com/weaviate/weaviate-python-client.git fd9dbdda75b64d59ec7742a0e95157a9a608a0de
  Running command git checkout -q fd9dbdda75b64d59ec7742a0e95157a9a608a0de
  Resolved https://github.com/weaviate/weaviate-python-client.git to commit fd9dbdda75b64d59ec7742a0e95157a9a608a0de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.0 -> 24.3.1
[n

You will need a weaviate-client version 4.9.6 or higher.

In [2]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.9.6.dev2+gfd9dbdda
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /opt/homebrew/lib/python3.11/site-packages
Requires: authlib, grpcio, grpcio-health-checking, grpcio-tools, httpx, pydantic, requests, validators
Required-by: langchain-weaviate, llama-index-vector-stores-weaviate, weaviate-cli


## Connect to Weaviate
You will need Weaviate v1.28 or higher.
You can run
````
docker run -p 8080:8080 -p 50051:50051 cr.weaviate.io/semitechnologies/weaviate:1.28.1
```


In [3]:
import weaviate, os

cohere_key = os.environ["COHERE_API_KEY"] # Replace with your Cohere key

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(
  headers={
    "X-COHERE-Api-Key": cohere_key
  }
)

# Option 2
# Connect to your Weaviate Client Service cluster
# client = weaviate.connect_to_wcs(
#     cluster_id="WCS-CLUSTER-ID", # Replace with your WCS cluster ID
#     auth_credentials=weaviate.AuthApiKey(
#       api_key="WCS-API-KEY" # Replace with your WCS API KEY
#     ),
#     headers={
#       "X-COHERE-Api-Key": cohere_key
#     }
# )

client.is_ready()

True

## Create a collection
> Collection stores your data and vector embeddings.

The collection below uses the "Kagome" tokenizer option to tokenize Japanese sentences.
(by adding `tokenization=wc.Tokenization.KAGOME_JA` to the properties)


* [How to set a tokenizer](https://weaviate.io/developers/weaviate/manage-data/collections#property-level-settings)
* [Available tokenizers](https://weaviate.io/developers/weaviate/config-refs/schema#tokenization)

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "ClothingItems", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("ClothingItems")):
    client.collections.delete("ClothingItems")

client.collections.create(
    name="ClothingItems",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_cohere( # specify the vectorizer and model type you're using
        model="embed-multilingual-v3.0", # Multi-lingual embedding model
        truncate="RIGHT", # defaults to RIGHT if not set 
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="item", 
                    data_type=wc.DataType.TEXT,
                    tokenization=wc.Tokenization.KAGOME_JA
), 
        wc.Property(name="description", 
                    data_type=wc.DataType.TEXT,
                    tokenization=wc.Tokenization.KAGOME_JA
),
    ]
)

print("Successfully created collection: ClothingItems.")

Successfully created collection: JeopardyQuestion.


## Import Data

In [5]:
data = [
    ["スポーツ用レギンス", "吸湿速乾性に優れた素材で作られたレギンス。ジムやヨガ、ランニングに最適です。"],
    ["点線模様の長靴", "雨の日のおしゃれにぴったりの長靴。ユニークな点線模様が特徴です。"],
    ["トレンチコート", "クラシックなデザインのトレンチコート。春や秋の軽い羽織りに最適な一着です。"],
    ["ひまわりの模様のドレス", "明るいひまわり柄が夏らしい印象のドレス。軽やかで涼しい素材が魅力です。"],
    ["ライトカーディガン", "肌寒い日の重ね着に便利なライトカーディガン。軽量で持ち運びにも便利です。"],
    ["フローラルミニスカート", "春や夏にぴったりの花柄ミニスカート。華やかなデザインで女性らしさを引き立てます。"],
    ["ピンクのスカート", "柔らかなピンク色がかわいらしいスカート。シンプルなデザインでコーディネートしやすいです。"],
    ["カシミアセーター", "高級感のあるカシミア素材で作られたセーター。寒い冬に暖かさとスタイルを提供します。"],
    ["ライトウールスカーフ", "秋や冬にぴったりのウールスカーフ。軽量で肌触りが良く、暖かさをプラスします。"],
    ["ニットドレス", "柔らかく暖かい素材で作られたニットドレス。寒い季節のカジュアルスタイルに最適です。"],
    ["ミディプリーツスカート", "エレガントなシルエットが特徴のプリーツスカート。どんなトップスとも相性抜群です。"],
    ["花柄の傘", "雨の日を明るくしてくれる花柄デザインの傘。軽量で使いやすい構造です。"],
    ["長袖の紺色のドレス", "落ち着いた紺色がエレガントな長袖ドレス。フォーマルな場にも最適です。"],
    ["春の花柄ドレス", "春の装いにぴったりの鮮やかな花柄ドレス。柔らかい素材で着心地も抜群です。"],
    ["ボア付きデニムジャケット", "冬の寒さから守るボア付きデニムジャケット。カジュアルスタイルにぴったりです。"],
    ["レザーアンクルブーツ", "シックで洗練されたデザインのレザーアンクルブーツ。秋冬の装いにおすすめです。"],
    ["防水ジャケット", "雨の日の外出を快適にする防水ジャケット。軽量で動きやすいデザインが魅力です。"],
    ["シンプルな黒のタートルネック", "どんなスタイルにも合わせやすい黒のタートルネック。秋冬の重ね着に最適です。"],
    ["シフォンブラウス", "軽やかで女性らしいデザインのシフォンブラウス。春や夏のフォーマルな場にもおすすめです。"]
]

# Get a collection object for "ClothingItems"
clothes = client.collections.get("ClothingItems")

with clothes.batch.dynamic() as batch:
    for d in data:
        # the call that performs data insert
        batch.add_object({
            "item": d[0],
            "description" : d[1],
        })

In [6]:
# Print the total number of imported chunks
print(clothes.aggregate.over_all(total_count=True).total_count)

19


In [7]:
response = clothes.query.fetch_objects(limit=3)

for item in response.objects:
    print(f"{item.properties['item']}: {item.properties['description']}")

春の花柄ドレス: 春の装いにぴったりの鮮やかな花柄ドレス。柔らかい素材で着心地も抜群です。
ライトウールスカーフ: 秋や冬にぴったりのウールスカーフ。軽量で肌触りが良く、暖かさをプラスします。
防水ジャケット: 雨の日の外出を快適にする防水ジャケット。軽量で動きやすいデザインが魅力です。


## Keyword Search

In [8]:
response = clothes.query.bm25(
    query="ドレス",
    limit=3
)

for item in response.objects:
    print(f"{item.properties['item']}: {item.properties['description']}")

長袖の紺色のドレス: 落ち着いた紺色がエレガントな長袖ドレス。フォーマルな場にも最適です。
ニットドレス: 柔らかく暖かい素材で作られたニットドレス。寒い季節のカジュアルスタイルに最適です。
ひまわりの模様のドレス: 明るいひまわり柄が夏らしい印象のドレス。軽やかで涼しい素材が魅力です。


## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [9]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
response = clothes.query.hybrid(
    query="春のドレス",
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print(f"{item.properties['item']}: {item.properties['description']}")

春の花柄ドレス: 春の装いにぴったりの鮮やかな花柄ドレス。柔らかい素材で着心地も抜群です。
ひまわりの模様のドレス: 明るいひまわり柄が夏らしい印象のドレス。軽やかで涼しい素材が魅力です。
フローラルミニスカート: 春や夏にぴったりの花柄ミニスカート。華やかなデザインで女性らしさを引き立てます。


# What next?
Weaviate's Japanese Kagome tokenizer is a significant step forward in helping Japanese developers build great AI applications.

Try out Weaviate, starting with the [Quickstart](https://weaviate.io/developers/weaviate/quickstart).

And where you have Japanese data, set the property tokenizer to "kagome_jp" as shown in the code above.

# Note
As of `1.28`, the tokenizer must be separately enabled by setting `ENABLE_TOKENIZER_KAGOME_JA` [environment variable](https://weaviate.io/developers/weaviate/config-refs/env-vars) to true. (For example, in the `docker-compose.yml` file.)